In [3]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
from google.cloud import bigquery

def fetch_data_by_soustype(project_id, dataset_id, table_id, soustype_name):
    """
    Fetch data from BigQuery for a specific sous-type.
    """
    client = bigquery.Client(project=project_id)
    query = f"""
    SELECT *
    FROM `{project_id}.{dataset_id}.{table_id}`
    WHERE soustype = @soustype_name
    """
    job_config = bigquery.job.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("soustype_name", "STRING", soustype_name)
        ]
    )
    query_job = client.query(query, job_config=job_config)
    return query_job.result().to_dataframe()

def cluster_data_by_soustype(project_id, dataset_id, table_id):
    """
    Cluster data by sous-type and then by coordinates, ensuring unique cluster labels per sous-type.
    """
    # Get unique sous-types
    client = bigquery.Client(project=project_id)
    query = f"SELECT DISTINCT soustype FROM `{project_id}.{dataset_id}.{table_id}`"
    soustypes_df = client.query(query).result().to_dataframe()

    clustered_data_list = []
    cluster_counter = 0  # Initialize a global cluster counter

    for soustype_name in soustypes_df['soustype']:
        # Fetch data for the current sous-type
        soustype_data = fetch_data_by_soustype(project_id, dataset_id, table_id, soustype_name)

        # Extract geographical coordinates
        soustype_data[['latitude', 'longitude']] = soustype_data['geo_point_2d'].str.split(',', expand=True).astype(float)

        # Filter data with valid coordinates
        valid_geo_data = soustype_data.dropna(subset=['latitude', 'longitude'])

        # Convert coordinates to radians
        earth_radius = 6371000  # Earth radius in meters
        valid_geo_data['lat_rad'] = np.radians(valid_geo_data['latitude'])
        valid_geo_data['lon_rad'] = np.radians(valid_geo_data['longitude'])

        # Apply DBSCAN on geographical coordinates
        coords = valid_geo_data[['lat_rad', 'lon_rad']]
        db = DBSCAN(eps=20/earth_radius, min_samples=1, algorithm='ball_tree', metric='haversine').fit(coords)

        # Add cluster labels to the DataFrame, offset by the current cluster counter
        valid_geo_data['cluster'] = db.labels_ + cluster_counter

        # Calculate the centroid for each cluster within the sous-type
        centroids = valid_geo_data.groupby('cluster').agg({
            'latitude': 'mean',
            'longitude': 'mean'
        }).reset_index()

        # Rename the centroid columns for clarity
        centroids.rename(columns={'latitude': 'centroid_latitude', 'longitude': 'centroid_longitude'}, inplace=True)

        # Merge the centroids back into the DataFrame
        valid_geo_data = valid_geo_data.merge(centroids, on='cluster', how='left')

        # Append the clustered data to the list
        clustered_data_list.append(valid_geo_data)

        # Update the cluster counter
        cluster_counter += len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)

    # Combine all clustered data into a single DataFrame
    clustered_data_full = pd.concat(clustered_data_list)

    # Print the shape of the DataFrame
    print("Shape of clustered_data_full:", clustered_data_full.shape)

    # Print the head of the DataFrame with centroids
    print("Head of clustered_data_full with centroids:")
    print(clustered_data_full.head())

    return clustered_data_full

# Example usage
clustered_data = cluster_data_by_soustype('conseil-quartier', 'dans_ma_rue', 'feed_requetes_dmr')


/opt/anaconda3/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
 

Shape of clustered_data_full: (1085468, 24)
Head of clustered_data_full with centroids:
   numero                                       type soustype  \
0   25203  Graffitis, tags, affiches et autocollants  Abribus   
1   16095  Graffitis, tags, affiches et autocollants  Abribus   
2    3722  Graffitis, tags, affiches et autocollants  Abribus   
3    4894  Graffitis, tags, affiches et autocollants  Abribus   
4    4858  Graffitis, tags, affiches et autocollants  Abribus   

                                         adresse code_postal     ville  \
0          2-6 Place de la Bastille, 75012 PARIS     75012.0  Paris 12   
1  50 rue du Faubourg Saint-Antoine, 75012 PARIS     75012.0  Paris 12   
2                    75 Rue de Lyon, 75012 PARIS     75012.0  Paris 12   
3                    44 Rue de Lyon, 75012 PARIS     75012.0  Paris 12   
4                   140 Rue de Lyon, 75012 PARIS     75012.0  Paris 12   

   arrondissement        conseilquartier   datedecl anneedecl  ...  \
0     

In [4]:
# Print the shape of the DataFrame
print("Shape of clustered_data:", clustered_data.shape)

# Print the head of the DataFrame with all columns
print("Head of clustered_data with all columns:")
print(clustered_data.head())


Shape of clustered_data: (1085468, 24)
Head of clustered_data with all columns:
   numero                                       type soustype  \
0   25203  Graffitis, tags, affiches et autocollants  Abribus   
1   16095  Graffitis, tags, affiches et autocollants  Abribus   
2    3722  Graffitis, tags, affiches et autocollants  Abribus   
3    4894  Graffitis, tags, affiches et autocollants  Abribus   
4    4858  Graffitis, tags, affiches et autocollants  Abribus   

                                         adresse code_postal     ville  \
0          2-6 Place de la Bastille, 75012 PARIS     75012.0  Paris 12   
1  50 rue du Faubourg Saint-Antoine, 75012 PARIS     75012.0  Paris 12   
2                    75 Rue de Lyon, 75012 PARIS     75012.0  Paris 12   
3                    44 Rue de Lyon, 75012 PARIS     75012.0  Paris 12   
4                   140 Rue de Lyon, 75012 PARIS     75012.0  Paris 12   

   arrondissement        conseilquartier   datedecl anneedecl  ...  \
0             

In [5]:
# count the number of clusters

clustered_data['cluster'].nunique()

311643

In [6]:

def create_clustering_table(clustered_data, project_id, dataset_id, new_table_id):
    """
    Create a new table in BigQuery with clustering data.
    """
    client = bigquery.Client(project=project_id)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(new_table_id)

    # Select only the necessary columns for the new table
    clustering_data = clustered_data[['id_dmr', 'cluster', 'centroid_latitude', 'centroid_longitude']]

    # Configure the load job
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",  # Overwrite the table if it exists
    )

    # Load the DataFrame to BigQuery
    load_job = client.load_table_from_dataframe(clustering_data, table_ref, job_config=job_config)
    load_job.result()  # Wait for the job to complete

    print(f"Clustering data uploaded to new table {dataset_id}.{new_table_id}")

# Example usage
create_clustering_table(clustered_data, 'conseil-quartier', 'dans_ma_rue', 'clustering_data')


/opt/anaconda3/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Clustering data uploaded to new table dans_ma_rue.clustering_data
